### Answering Descriptive and Exploratory Questions About my Project

In [1]:
# Import packages
import igraph as ig
import os
from subprocess import Popen, PIPE

# Initializing dataset names
dnames = list(['MRN114', 'KKI2009', 'SWU4'])
print "Datasets: " + ", ".join(dnames)
print "D = " + str(len(dnames))

# Getting graph names
a = list()
for dd in dnames:
        a.extend([root+'/'+file for root, dir, files in os.walk(dd) for file in files])

Datasets: MRN114, KKI2009, SWU4
D = 3


**Descriptive** <br />
- *What is N<sub>i</sub> for all i?*

In [2]:
# Get lengths of sublists and total list

print "N_i for each dataset (same order as above): " +\
      ", ".join([str(len(filter(lambda x: dd in x, a))) for dd in dnames])
print "Total N = " + str(len(a))


N_i for each dataset (same order as above): 114, 42, 454
Total N = 610


- *What is |V|?*

In [3]:
# We know that |V| is the same for all graphs, so here we really only need to load in 1
graph = ig.Graph.Read_GraphML(a[0])
print "|V| = " + str(graph.vcount())

|V| = 70


- *Are there graphs G<sub>n<sub>i</sub></sub> that are expected but not present?*

In [ ]:
# This is a tricky one, since we only know our data based on the directory search...
# How about let's check if every file that appears does successfully load as a graph
# (this is a lot of overhead, as I'm loading 610 graphs, but I'll need them anyways
# so I'll let it slide)


- *Do the graphs G<sub>n<sub>i</sub></sub> contain any values of A that cannot be processed traditionally (i.e. inf, NaN)?*

- *How sparse, |E<sub>n<sub>i</sub></sub>| / |V<sub>n<sub>i</sub></sub>|x|V<sub>n<sub>i</sub></sub>|, are the graphs?*

**Exploratory** <br />
- *What is mean(|E|) for each dataset i?*

- *What is the average graph, where average here means the graph consiting of edges and weights corresponding to the average weight of a given potential edge across the all the datasets?*

- *What is the distribution of max(A)-min(A) (i.e. dynamic range) for each dataset i?*